## Using Mito and Python to send an email

This notebook uses Mito to import data, create a pivot table, and send an email with the pivot table embedded in it. 
It sends the email using Outlook and utilizes html styling supported by Outlook.

### Import Data

In [65]:
import pandas as pd
df = pd.DataFrame({'City': ['New York City', 'New York City', 'Philadelphia', 'Washington D.C.', 'Boston', 'Boston'], 'Ticket_Office': ['N', 'Y', 'N', 'Y', 'N', 'Y']}) 

### Create pivot table

In [66]:
from mitosheet import *; # Analysis Name:id-jwatoqmnru;
    
# Pivoted into df
tmp_df = df[['City']]
pivot_table = tmp_df.pivot_table(
    index=['City'],
    values=['City'],
    aggfunc={'City': ['count']}
)
pivot_table.set_axis([flatten_column_header(col) for col in pivot_table.keys()], axis=1, inplace=True)
df_pivot = pivot_table.reset_index()

# Renamed columns Number of Stations
df_pivot.rename(columns={'City count': 'Number of Stations'}, inplace=True)


### Apply simple styling to the dataframe

In [67]:
df_pivot_styler = df_pivot.style\
    .set_table_styles([
        {'selector': 'thead', 'props': [('color', '#FFFFFF'), ('background-color', '#4D73BE')]},
])


### Generate and Send the email

In [68]:
import win32com.client

# Generate a new email
outlook = win32com.client.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'aaron@sagacollab.com'
mail.Subject = 'Mito Email Template'

# Add the pivot table to the email body
mail.HTMLBody = table_html = df_pivot_styler.to_html()

# Send the email
mail.Send()